In [1]:
import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [3]:
Delete_list_trials = {}
t_sup, b_s, a_s = 280, 5, 15

opt_base = {'stop_search_misac': None, 'time_sup': t_sup,
            'before_sacc': b_s, 'after_sacc': a_s} 

for x in range(len(sujet)) :
    
    timeStr, observer = time[x], sujet[x]
    mode = 'enregistrement'
    #print(observer,timeStr)
    
    file = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.pkl')
    with open(file, 'rb') as fichier : exp = pickle.load(fichier, encoding='latin1')

    A = ANEMO(exp)
    
    resultats = os.path.join('..','data', mode + '_' + observer + '_' + timeStr + '.asc')
    data = read_edf(resultats, 'TRIALID')

    N_trials, N_blocks = exp['N_trials'], exp['N_blocks']
    
    Delete_list_trials['%s'%observer] = []
    for block in range(N_blocks) :
        supp_trial = []
        for trial in range(N_trials) :
            
            trial_data = trial + N_trials*block
            arg = A.arg(data[trial_data], trial=trial, block=block)
            opt = opt_base.copy()
            opt.update(arg)
            
            velocity_NAN = A.velocity_NAN(**opt)
            new_sacc = arg.saccades
            s_gap = []
            for s in range(len(new_sacc)) :
                s_gap.append([])
                for x_data in np.arange((new_sacc[s][0]-arg.t_0-b_s), (new_sacc[s][1]-arg.t_0+a_s)) :
                    if x_data in np.arange(arg.TargetOn-arg.t_0-150, arg.TargetOn-arg.t_0+150) :
                        s_gap[s].append(x_data)

            t = None
            
            # supp grosse saccade GAP
            for x in range(len(s_gap)) :
                if len(s_gap[x]) > 45 :
                    if t is None :
                        supp_trial.append(trial)
                        t = True
            
            # supp trop saccade poursuite
            if t is None :
                lala2 = velocity_NAN[arg.TargetOn-arg.t_0:arg.TargetOff-arg.t_0-300]
                baba2 = [x for x in lala2 if str(x)=='nan']
                if len(baba2) >= len(lala2)/3.6 :
                    supp_trial.append(trial)
        
        #print(block, supp_trial)
        Delete_list_trials['%s'%observer].extend([supp_trial])

/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:444: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.5/dist-packages/ANEMO-0.0.1-py3.5.egg/ANEMO/ANEMO.py:444: RuntimeWarning: invalid value encountered in true_divide


In [4]:
a = '../parametre/Delete/Delete_list_BadTrials_Full_fct_velocity_sigmo.pkl'
with open(a, 'rb') as fichier :
    b = pickle.load(fichier, encoding='latin1')
    list_delete = b

In [5]:
nb_delete = 0
nb_fct = 0
nb_main = 0
for s in list_delete.keys():
    for b in range(len(list_delete[s])) :
        nb_delete += len(set(list_delete[s][b]))
        nb_fct += len(set(Delete_list_trials[s][b]))
        
        for t in list_delete[s][b]:
            if t not in Delete_list_trials[s][b] :
                nb_main+=1
            

In [6]:
nb_trial = 12*3*200
print(nb_trial)

7200


In [7]:
print(nb_delete, (nb_delete/nb_trial)*100)

548 7.611111111111112


In [8]:
print(nb_fct, (nb_fct/nb_trial)*100, (nb_fct/nb_delete)*100)

314 4.361111111111112 57.299270072992705


In [9]:
print(nb_main, (nb_main/nb_trial)*100, (nb_main/nb_delete)*100)

234 3.25 42.700729927007295
